In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_9836/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
    
    def has_minimum_requirements(self, df, sort_by="r2", boundary=0.75):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [6]:
analize = Analizer()
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
0,model_2_7_0,0.927535,0.919725,0.916506,0.918487,0.080197,0.119160,0.087316,0.104175,"Hidden Size=[21, 10], regularizer=0.1, learnin..."
1,model_2_7_1,0.926530,0.880671,0.924329,0.897485,0.081309,0.177133,0.079134,0.131016,"Hidden Size=[21, 10], regularizer=0.1, learnin..."
6,model_2_7_5,0.916162,0.830264,0.869526,0.845386,0.092784,0.251956,0.136446,0.197599,"Hidden Size=[21, 10], regularizer=0.1, learnin..."
8,model_2_7_6,0.915978,0.831958,0.861041,0.843160,0.092988,0.249442,0.145320,0.200444,"Hidden Size=[21, 10], regularizer=0.1, learnin..."
46,model_3_8_2,0.899693,0.859530,0.889042,0.881634,0.111010,0.188455,0.175548,0.182381,"Hidden Size=[21, 10], regularizer=0.02, learni..."
...,...,...,...,...,...,...,...,...,...,...
192,model_8_7_21,0.852353,0.865375,0.821821,0.844929,0.163402,0.174153,0.287725,0.227599,"Hidden Size=[40, 15], regularizer=0.02, learni..."
195,model_8_7_20,0.850982,0.866207,0.822845,0.845848,0.164919,0.173076,0.286072,0.226251,"Hidden Size=[40, 15], regularizer=0.02, learni..."
198,model_8_7_19,0.849587,0.866978,0.823923,0.846766,0.166462,0.172079,0.284331,0.224904,"Hidden Size=[40, 15], regularizer=0.02, learni..."
200,model_8_7_18,0.848131,0.867672,0.824977,0.847635,0.168074,0.171182,0.282629,0.223627,"Hidden Size=[40, 15], regularizer=0.02, learni..."
